In [ ]:
import pandas as pd
import numpy as np

In [ ]:
cols = ['timestamp', 'hostname', 'record', 'TTL','DNS_IP','ANS_IP', '2LD']
df = pd.read_csv('data/raw_logs_auth_2016_sep_12_00_14', sep='\t', names= cols, nrows=5500000)
df = df.sort_values(['timestamp'])
df['tunnel'] = 0

In [ ]:
# get the count of A records as those are good indicators of tunneling
t = df[df['record'] != 'A']
suspect_domains = t.groupby(['2LD']).size().sort_values(ascending=False)

In [ ]:
domains = suspect_domains.to_frame().index.tolist()[:1000]

In [ ]:
for domain in domains:
    print domain
    # limit the df to the suspected domain
    domain_df = df[(df['2LD'] == domain)]
    grp_df = domain_df.groupby(['record', 'timestamp']).size().to_frame()
    grp_df = grp_df.reset_index()
    
    #
    grp_df['delta_size'] = np.abs(grp_df[0] - grp_df[0].shift(1))
    
    # get the time difference
    grp_df['delta_time'] = grp_df['timestamp'] - grp_df['timestamp'].shift(1) 
    rec_time = grp_df.groupby(['record']).median()
    rec_type = rec_time.index.values
    time = rec_time[['delta_time']].values.reshape(1,-1)[0].tolist()
    m_time = zip(rec_type, time)
    
    for r,dt in m_time:
        sus_df = grp_df[(grp_df['record'] == r) & (grp_df['delta_time'] > dt) 
                        & (grp_df['delta_size'] > 100)]
        if len(sus_df) > 0:
            ts = sus_df['timestamp'].values
            if len(ts) > 2:
                for i in range(0,len(ts)):
                    if i < len(ts)-1:
                        sus = domain_df[(domain_df['timestamp'] >= ts[i]) & 
                                  (domain_df['timestamp'] < ts[i+1]) & 
                                  (domain_df['record'] == r) ]
                        df.set_value(sus.index, 'tunnel',1)

In [ ]:
df = df[~df['hostname'].str.contains("apple.com")]

df = df[~df['hostname'].str.contains("google.com")]

df = df[~df['hostname'].str.contains("yahoo.com")]

df = df[~df['hostname'].str.contains("dropbox.com")]

df = df[~df['hostname'].str.contains("twitter.com")]

df = df[~df['hostname'].str.contains("vimeo.come")]

In [ ]:
#tunnels_df = df[df['tunnel'] == 1] akadns.net. #throw thsi index out 3453857
df[df ['tunnel'] == 0]

## Summary
Detecting DNS tunneling in this exercise relied mostly on statistical measures. One of the most noted indication of tunneling is observing the type of DNS record-- A, CNAME, MX, or TXT.  Other papers have shown that it is possible to tunnel A and CNAME records, traditionally, TXT records have been a more positive indicator. 

We start with counting the type of records, where at the 2LD domains with a high count of CNAME or MX records we will examine closer. With each suspicious domain, we get the frequency of requests per timestamp. We expect an increased in requests during tunneling, and with a high latency between timestamps. To observe this, we count the number of requests of a domain at a given time. The first check is to see a high latency between requests. This is an indicator that the server observed some sort of problem whether it was from the many request calls or the start of a tunneling. 

We also observe that the authorative IP response is quite consistent (returning from the same range of family addresses and 4-5 unique authorative IPs) which we expect for a tunnel would attempt to go to the same authorative IP. Many of hostnames in that time frame exhibit qualities of tunneling such as the lack of non-English words and a high frequency of numerical characters in the name. We also notice that there this cyclic rounds of increased then decreased requests occur at time 1473647400.   